In [1]:
import netCDF4 as nc
import numpy as np
import xarray as xr
import warnings
from eofs.xarray import Eof
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import gridspec
import datetime
import seaborn as sns
import cmocean as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy.stats import linregress
import matplotlib.patches as mpatches
import scipy.io as sio
import matplotlib.colors as mcolors
import matplotlib.patches as patch
import sys
import os
sys.path.append(os.path.abspath('../scripts'))
from proj_utils import *
fig_path      = '../figures'

In [2]:
# --- Pull json velocity files ---
s3, credentials = init_S3FileSystem(use_earthdata=False, requester_pays=True)
mzz_local_directory = Path('/efs_ecco/mzz-jsons-V4r5/MZZ_mon_mean_native')

mzz_local_file= mzz_local_directory / 'OCEAN_VELOCITY_mon_mean_native_llc090_ECCOV4r5.json' # Pull monthly velocity 
fs = fsspec.filesystem("reference",     
                       fo=str(mzz_local_file),
                       remote_protocol="s3",
                       remote_options={"anon":False, "requester_pays":True},
                       skip_instance_cache=True)

fs.asynchronous = True
store = zarr.storage.FsspecStore(fs)
ds = xr.open_dataset(store, engine='zarr',
                     consolidated=False, chunks={'time':4, 'Z':50})

In [3]:
# --- Pull geometry file ---
bathy_path = user_home_dir + '/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc'
ds_bathy   = xr.open_dataset(bathy_path).Depth
ds_geom    = xr.open_dataset(bathy_path)

In [4]:
# --- Get (full) north face
ds = get_na_tile(ds)
ds = subset_tgb_box(ds)

ds_geom = get_na_tile(ds_geom)
ds_geom = subset_tgb_box(ds_geom)

ds_north_face = ds.sel(i = slice(ds.i[0]), i_g = slice(ds.i_g[0])).squeeze()
ds_geom_north = ds_geom.sel(i = slice(ds.i[0]), i_g = slice(ds.i_g[0])).squeeze()

In [ ]:
# --- Calculate flux, NORTH FACE FULL ---
u    = ds_north_face['UVEL'].squeeze()
hfac = ds_geom_north['hFacW'].squeeze()
dy   = ds_geom_north['dyC'].squeeze()
drf  = ds_geom_north['drF'].squeeze()

drf_3d = drf.broadcast_like(u)
dx_3d  = dy.broadcast_like(u)

area = dx_3d * drf_3d * hfac

flux_north = (u * area).sel(j_g = 72).squeeze()

flux_north_mn = flux_north.mean(dim = 'time').compute()

In [ ]:
# --- Calculate NORTH FACE flux time series, just DWBC box ---
flux_north_masked = flux_north[:,18:40,5:10]
flux_north_dwbc   = flux_north_masked.sum(dim = ('k','j'))/1e6

In [ ]:
# --- Get (full) west face 
ds_east_face = ds.sel(j = slice(ds.j[0]-0.25,ds.j[0]+0.25), j_g = slice(ds.j_g[0]-0.25,ds.j_g[0]+0.25)).squeeze()
ds_geom_east = ds_geom.sel(j = slice(ds.j[0]-0.25,ds.j[0]+0.25), j_g = slice(ds.j_g[0]-0.25,ds.j_g[0]+0.25)).squeeze()

In [ ]:
# --- Calculate flux, WEST FACE FULL ---
v    = ds_east_face['VVEL'].squeeze()
hfac = ds_geom_east['hFacS'].squeeze()
dx   = ds_geom_east['dxC'].squeeze()
drf  = ds_geom_east['drF'].squeeze()

drf_3d = drf.broadcast_like(v)
dx_3d  = dx.broadcast_like(v)

area = dx_3d * drf_3d * hfac

flux_east = (v * area).sel(i_g = 42)

flux_east_mn = flux_east.mean(dim = 'time').compute()

In [ ]:
# --- Calculate EAST FACE flux time series, just DWBC box ---
flux_east_masked = flux_east[:,25:45,7:13]
flux_east_dwbc   = flux_east_masked.sum(dim = ('k','i'))/1e6

In [ ]:
# --- Calculate correlation coefficient ---
corr_coef = np.corrcoef(flux_north_masked.sum(dim = ('k','j'))/1e6,-flux_east_masked.sum(dim = ('k','i'))/1e6)[0,1]

In [ ]:
# --- Plot flux time series ---
sv_name = '/north_face_east_face_flux_ts'
sns.set_style('whitegrid',{"grid.linestyle": ":"})
fig     = plt.figure(figsize=(12, 6))

north_face  = plt.plot(flux_north_masked.time,flux_north_dwbc, c = '#dc267f', alpha = 0.5, label = 'Flux through North Face')
east_face  = plt.plot(flux_east_masked.time,-flux_east_dwbc, c = '#648fff', alpha = 0.5, label = 'Flux through East Face')

mean_flux   = plt.axhline(flux_north_mask.sum(dim = ('k','j')).mean(dim='time').compute()/1e6, color = '#dc267f', linestyle = '--', linewidth = 2)
mean_flux   = plt.axhline(-flux_east_mask.sum(dim = ('k','i')).mean(dim='time').compute()/1e6, color = '#648fff', linestyle = '--', linewidth = 2)
xtix = plt.yticks(fontsize=18)
xtix = plt.xticks(fontsize=18)

plt.xlim(flux_north.time[0],flux_north.time[-1])
plt.title('r = ' + str('{:.2f}'.format(np.corrcoef(flux_north_mask.sum(dim = ('k','j'))/1e6,-flux_east_mask.sum(dim = ('k','i'))/1e6)[0,1])), fontsize = 18)
plt.legend(fontsize=15)
plt.ylabel('Volume Flux of DWBC [$Sv$]',fontsize = 18 )
plt.savefig(fig_path + sv_name + '.png', format='png', bbox_inches="tight",dpi=500)

In [ ]:
# --- Plot ratio of flux in to out ---
sv_name = '/north_face_east_face_ratio_ts'
sns.set_style('whitegrid',{"grid.linestyle": ":"})
fig     = plt.figure(figsize=(12, 6))

north_face  = plt.plot(flux_east.time, (abs(flux_east_dwbc)/abs(flux_north_dwbc)*100), alpha = 0.5, c = '#785ef0')

mean_flux   = plt.axhline((abs(flux_east_dwbc)/abs(flux_north_dwbc)*100).mean(), color = '#785ef0', linestyle = '--', linewidth = 2)
xtix = plt.yticks(fontsize=18)
xtix = plt.xticks(fontsize=18)

plt.xlim(flux_north.time[0],flux_north.time[-1])
#plt.title('Ratio of Flux Out to Flux In ', fontsize = 18)
plt.ylabel('Flux East Face/Flux North Face [%]',fontsize = 18 )
plt.savefig(fig_path + sv_name + '.png', format='png', bbox_inches="tight",dpi=500)

In [ ]:
# --- Calculate north and west face spectra
sys.path.append(os.path.abspath('../scripts'))
from proj_utils import *
from analysis_helpers import *

north_freq, psd, north_spectrum = calc_flux_spec(flux_north_dwbc)
east_freq, psd, east_spectrum  = calc_flux_spec(flux_east_dwbc)

In [ ]:
# ---  Plot of north and west face spectra ---
sv_name = '/north_face_east_face_specs'
fig     = plt.figure(figsize=(8, 6))
plt.xscale('log')
plt.yscale('log')
periods = np.array([0.25, 0.5, 1, 2, 5, 10, 20, 50])  # years
frequencies = 1 / periods
frequencies = frequencies[(frequencies >= freq.min()) & (frequencies <= freq.max())]
period_labels = [f'{1/f:.2f}' for f in frequencies]  # years

plt.plot(north_spec.frequency,north_spec,c = '#dc267f')
plt.plot(east_spec.frequency,east_spec,c = '#648fff')

plt.xticks(frequencies, labels=period_labels,fontsize = 15)
plt.yticks(fontsize = 15)



plt.xlabel('Frequency [$yr^{-1}$]',fontsize = 15)
plt.ylabel('PSD',fontsize = 15)
plt.savefig(fig_path + sv_name + '.png', format='png', bbox_inches="tight",dpi=500)